In [65]:
import pandas
df = pandas.read_csv('academy_awards.csv', encoding='ISO-8859-1')

In [66]:
df.head(3)

,Year,Category,Nominee,Additional Info,Won?,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,2010 (83rd),Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},NO,NaN,NaN,NaN,NaN,NaN,NaN
1,2010 (83rd),Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},NO,NaN,NaN,NaN,NaN,NaN,NaN
2,2010 (83rd),Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},NO,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
df['Unnamed: 6'].value_counts()

*                                                                   9
 flexibility and water resistance                                   1
 direct radiator bass style cinema loudspeaker systems. [Sound]"    1
 sympathetic                                                        1
Name: Unnamed: 6, dtype: int64

In [68]:
df['Additional Info'].value_counts()

Metro-Goldwyn-Mayer                                                                           60
Walt Disney, Producer                                                                         57
Warner Bros.                                                                                  42
John Williams                                                                                 37
France                                                                                        35
Alfred Newman                                                                                 34
Italy                                                                                         26
Paramount                                                                                     24
Gordon Hollingshead, Producer                                                                 22
Edith Head                                                                                    22
20th Century-Fox              

### Filter Category

#### Clean up the Year

In [69]:
df['Year'] = df['Year'].str[0:4]

In [70]:
df['Year'] = df['Year'].astype('int64')

In [71]:
later_than_2000 = df[df['Year']>2000]

In [72]:
award_categories=['Actor -- Leading Role', 'Actor -- Supporting Role', 'Actress -- Leading Role','Actress -- Supporting Role']

In [73]:
nominations = later_than_2000[later_than_2000["Category"].isin(award_categories)]

In [74]:
final_nominations = nominations.copy()

#### Create new boolean field 'Won' (0/1 values)

In [75]:
replace_dict = { 'YES': 1, 'NO': 0 }
final_nominations['Won']= final_nominations['Won?'].map(replace_dict)

In [76]:
final_nominations.drop('Won?',axis=1,inplace=True)
final_nominations.drop('Unnamed: 5',axis=1,inplace=True)
final_nominations.drop('Unnamed: 6',axis=1,inplace=True)
final_nominations.drop('Unnamed: 7',axis=1,inplace=True)
final_nominations.drop('Unnamed: 8',axis=1,inplace=True)
final_nominations.drop('Unnamed: 9',axis=1,inplace=True)
final_nominations.drop('Unnamed: 10',axis=1,inplace=True)

#### Cleaning Up The Additional Info Column

In [77]:
additional_info_one = final_nominations['Additional Info'].str.rstrip("'}")
additional_info_two = additional_info_one.str.split(" {'")
movie_names = additional_info_two.str[0]
characters = additional_info_two.str[1]

In [78]:
final_nominations['Movie'] = movie_names

In [79]:
final_nominations['Character'] = characters

In [80]:
final_nominations.drop('Additional Info',axis=1,inplace=True)

### Write to SQLite db 

In [81]:
import sqlite3
conn = sqlite3.connect('nominations.db')

In [82]:
final_nominations.to_sql("nominations", conn, if_exists='replace', index=False, index_label=None, chunksize=None, dtype=None)

In [84]:
conn.execute('pragma table_info(nominations);').fetchall()

[(0, 'Year', 'INTEGER', 0, None, 0),
 (1, 'Category', 'TEXT', 0, None, 0),
 (2, 'Nominee', 'TEXT', 0, None, 0),
 (3, 'Won', 'INTEGER', 0, None, 0),
 (4, 'Movie', 'TEXT', 0, None, 0),
 (5, 'Character', 'TEXT', 0, None, 0)]